# Створення нейронної мережі

У цьому завданні ми створимо повнозв'язну нейронну мережу, використовуючи при цьому низькорівневі механізми tensorflow.

Архітектура нейромережі представлена на наступному малюнку. Як бачиш, у ній є один вхідний шар, два приховані, а також вихідний шар. В якості активаційної функції у прихованих шарах буде використовуватись сигмоїда. На вихідному шарі ми використовуємо softmax.

Частина коду зі створення мережі вже написана, тобі потрібно заповнити пропуски у вказаних місцях.

## Архітектура нейронної мережі

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>


## Про датасет MNIST

Дану нейромережу ми будемо вивчати на датасеті MNIST. Цей датасет являє собою велику кількість зображень рукописних цифр розміром $28 \times 28$ пікселів. Кожен піксель приймає значення від 0 до 255.

Як і раніше, датасет буде розділений на навчальну та тестову вибірки. При цьому ми виконаємо нормалізацію всіх зображень, щоб значення пікселів знаходилось у проміжку від 0 до 1, розділивши яскравість кожного пікселя на 255.

Окрім того, архітектура нейронної мережі очікує на вхід вектор. У нашому ж випадку кожен об'єкт вибірки являє собою матрицю. Що ж робити? У цьому завданні ми "розтягнемо" матрицю $28 \times 28$, отримавши при цьому вектор, що складається з 784 елементів.

![MNIST Dataset](https://www.researchgate.net/profile/Steven-Young-5/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

Більше інформації про датасет можна знайти [тут](http://yann.lecun.com/exdb/mnist/).

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras as K
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [2]:
num_classes = 10 # загальна кількість класів, у нашому випадку це цифри від 0 до 9
num_features = 784 # кількість атрибутів вхідного вектора 28 * 28 = 784

learning_rate = 0.001 # швидкість навчання нейронної мережі
training_steps = 3000 # максимальне число епох
batch_size = 256 # перераховувати ваги мережі ми будемо не на всій вибірці, а на її випадковій підмножині з batch_size елементів
display_step = 100 # кожні 100 ітерацій ми будемо показувати поточне значення функції втрат і точності

n_hidden_1 = 128 # кількість нейронів 1-го шару
n_hidden_2 = 256 # кількість нейронів 2-го шару

In [3]:
# from tensorflow.keras.datasets import mnist
from keras.datasets import mnist

# Завантажуємо датасет
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Перетворюємо цілочисельні пікселі на тип float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Перетворюємо матриці розміром 28x28 пікселів у вектор з 784 елементів
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Нормалізуємо значення пікселів
x_train, x_test = x_train / 255., x_test / 255.

# Перемішаємо тренувальні дані
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))  #  Створюється Dataset з тензорів "x_train" і "y_train"
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
# train_data.repeat(): додає повторення епох нескінченну кількість разів
# shuffle(5000): виконується перемішування даних в Dataset
# batch(batch_size): групує дані в батчі (пакети) розміру "batch_size"
# prefetch(1): додає можливість попереднього завантаження даних. В даному випадку, 1 означає, що буде завантажено один батч даних в оперативну пам'ять

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# Створимо нейронну мережу

class DenseLayer(tf.Module):
    # Шар для роботи з вагами та зсувами
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)

        self.w = tf.Variable(
            tf.random.normal([in_features, out_features]), name="w"
        )  # задається тензор (матрицю) ваг з випадковими значеннями, взятими з нормального розподілу (розподіл Гаусса) розміром [in_features, out_features]
        # in_features - визначає кількість входів для кожного нейрона, out_features - кількість нейронів у шарі
        # tf.Variable(...): використовується для створення змінної TensorFlow, яку можна змінювати під час навчання моделі. Ваги мережі потрібно оновлювати під час процесу навчання
        # name="w": аргумент, який визначає ім'я цієї змінної, що може бути використане для отримання доступу до неї та ідентифікації її під час візуалізації моделі

        self.b = tf.Variable(tf.zeros([out_features]), name="b")
        # tf.zeros([out_features]): створює тензор (масив) розміром [out_features], в якому всі значення встановлені на нуль

    def __call__(self, x, activation=0):
        # метод __call__ представляє собою операцію, яка викликається, коли екземпляр класу об'єкта використовується як функція. При виклику шару передається вхід x, а також параметр activation
        y = tf.matmul(x, self.w) + self.b  # операція матричного множення та подальшого додавання елементів вектора зсуву
        if activation != 0:
            return tf.nn.softmax(y) # перетворення виходів у ймовірністі, які можуть інтерпретуватися як ймовірність приналежності до кожного класу
        else:
            return tf.nn.sigmoid(y)  # поверне новий тензор з такими ж розмірами, як і вхідний, але кожен його елемент буде відповідати значенню сигмоїд-функції для відповідного елементу вхідного тензору

class NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    # Перший шар, який складається з 128 нейронів
    self.layer1 = DenseLayer(in_features=num_features, out_features=n_hidden_1, name="layer1")

    # Другий шар, який складається з 256 нейронів
    self.layer2 = DenseLayer(in_features=128, out_features=256, name="layer2")

    # Вихідний шар
    self.output_layer = DenseLayer(in_features=256, out_features=num_classes, name="output_layer")


  def __call__(self, x):
    # Передача даних через перші два шари та вихідний шар з функцією активації softmax
    x = self.layer1(x, activation=1)
    x = self.layer2(x, activation=1)
    output = self.output_layer(x, activation=1)

    return output

In [ ]:
# В якості функції помилки в даному випадку зручно взяти крос-ентропію

# Обчислення середніх крос-ентропійних втрат (cross-entropy loss) між one-hot закодованими мітками y_true та передбаченими ймовірностями y_pred
def cross_entropy(y_pred, y_true):
    # Закодувати label в one hot vector
    y_true = tf.one_hot(y_true, depth=num_classes)
    # y_true - вектор міток класів
    # tf.one_hot - перетворення вектора y_true у матрицю one_hot, де кожен рядок є одним зразком даних,
    # а стовпці відповідають унікальним класам. Якщо зразок належить класу, то відповідний стовпець міститиме 1, а решта - 0

    # Значення передбачення, щоб уникнути помилки log(0).
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # y_pred - передбаченні значення моделі (в межах від 0 до 1)
    # 1e-9 - мале число, додається для того, щоб уникнути ділення на нуль. Тут воно використовується як нижня межа для обмеження
    # 1.0 - верхня межа для обмеження. Якщо значення y_pred більше 1, вони будуть замінені на 1
    # tf.clip_by_value - обмежує значення y_pred між вказаними межами. Всі значення, менше 1e-9, будуть встановлені на 1e-9, а всі значення, більше 1, будуть встановлені на 1

    # Обчислення крос-ентропії
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    # y_true - one-hot закодовані мітки класів
    # tf.math.log(y_pred) - обчислює логарифм ймовірностей
    # y_true * tf.math.log(y_pred) - поелементне множення one-hot закодованих міток класів на логарифми ймовірностей
    # -tf.reduce_sum - обчислює суму всіх значень, а знак мінус вказує на те, що ми обчислюємо зворотню величину (втрати), а не значення функції корисності
    # tf.reduce_mean - обчислює середнє значення втрат для всіх прикладів у партії

# Як метрику якості використовуємо точність
# Обчислимо її як відношення правильно класифікованих екземплярів до загальної кількості екземплярів (як середнє арифметичне точності для кожного прикладу в батчі)
def accuracy(y_pred, y_true):
    # Отримаємо індекс максимального значення для кожного класу
    predicted_classes = tf.argmax(y_pred, axis=1)
    # y_pred - передбаченні значення моделі (в межах від 0 до 1)
    # tf.argmax - отримуємо індекс класу (максисимальне значення), який модель вважає найбільш ймовірним
    # axis=1 - максимальне значення шукається вздовж другої осі, тобто вздовж стовпців

    # Отримаємо індекси правильних класів
    true_classes = tf.argmax(y_true, axis=1)
    # y_true - one-hot закодовані мітки класів

    # Порівняємо передбачені та правильні класи
    correct_predictions = tf.equal(predicted_classes, true_classes)
    # tf.equal - порівнює елементи двох тензорів та повертає булевий тензор, де True вказує на правильне передбачення

    # Обчислимо середню точність
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, dtype=tf.float32))
    # tf.cast - використовується для перетворення булевого тензора correct_predictions на тензор з числовими значеннями (0 та 1)
    # tf.reduce_mean - обчислює середнє значення цього тензора, що відображає середню точність для всіх екземплярів

    return accuracy

In [ ]:
# Створимо екземпляр нейронної мережі
neural_net = NN(name="mnist")

# Функція навчання нейромережі
def train(neural_net, input_x, output_y):
  # Для налаштування вагів мережі будемо використовувати стохастичний градієнтний спуск
  optimizer = tf.optimizers.SGD(learning_rate)

  # Активація автоматичного диференціювання
  with tf.GradientTape() as g:
    pred = neural_net(input_x)
    loss = cross_entropy(pred, output_y)

    # Отримаємо список оптимізованих параметрів
    # Місце для вашого коду
    # Обчислимо за ними значення градієнта
    # Місце для вашого коду

    # Модифікуємо параметри
    # Місце для вашого коду

In [ ]:
# Тренування мережі

loss_history = []  # кожні display_step кроків зберігай в цьому списку поточну помилку нейромережі
accuracy_history = [] # кожні display_step кроків зберігай в цьому списку поточну точність нейромережі

# У цьому циклі ми будемо проводити навчання нейронної мережі
# із тренувального датасета train_data вилучи випадкову підмножину, на якій
# відбудеться тренування. Використовуй метод take, доступний для тренувального датасета.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Оновлюємо ваги нейронної мережі
    # Місце для вашого коду

    if step % display_step == 0:
        pred = neural_net(batch_x)
        # Місце для вашого коду


In [ ]:
# Виведіть графіки залежності зміни точності і втрат від кроку
# Якщо все зроблено правильно, то точність повинна зростати, а втрати зменшуватись

import matplotlib.pyplot as plt

# Виведіть графік функції втрат
# Місце для вашого коду

# Виведіть графік точності
# Місце для вашого коду


In [ ]:
# Обчисліть точність навченої нейромережі
# Місце для вашого коду
# Тестування моделі на тестових даних
# Місце для вашого коду

In [ ]:
# Протестуйте навчену нейромережу на 10 зображеннях. З тестової вибірки візьміть 5
# випадкових зображень і передайте їх у нейронну мережу.
# Виведіть зображення та випишіть  поруч відповідь нейромережі.
# Зробіть висновок про те, чи помиляється твоя нейронна мережа, і якщо так, то як часто?

# Місце для вашого коду


